In [8]:
import pandas as pd 
import numpy as np 

import warnings
warnings.filterwarnings('ignore')

1. 환율 데이터 - 나라별 환율
   - 주말 데이터 없음
2. 부품 가격 데이터
3. 전처리
   * 국가별 관세 비율은 다음과 같다. 미국(25%), 인도(5%), 유럽(10%), 일본(3%)
   * 일자별 환율은 공휴일 및 주말은 발표되지 않는다.
   * 공휴일 및 주말에 구매한 건(환율 데이터에 날짜가 없는 경우)에 대한 환율 계산은 13:00 이전에는 전날의 환율을, 13:00 이후에는 다음날의 환율을 적

In [91]:
df =pd.read_csv('https://raw.githubusercontent.com/doeungim/ADP_DE2/refs/heads/main/ADP_EXAM_PRAC/EXAM_3/m1.csv')
ex =pd.read_csv('https://raw.githubusercontent.com/doeungim/ADP_DE2/refs/heads/main/ADP_EXAM_PRAC/EXAM_3/m1_exchange.csv')

In [92]:
# 환율 데이터 처리 
ex['일자'] = pd.to_datetime(ex['일자'])
# 나라별 환율 날짜 확인 
print(ex.groupby(['type']).agg(mi = ('일자', min), ma = ('일자', max)))

                mi         ma
type                         
eur-krw 2020-04-13 2025-04-14
inr-krw 2020-04-13 2025-04-14
jpy-krw 2020-04-13 2025-04-14
usd-krw 2020-04-13 2025-04-14


In [104]:
# 환율 데이터 전처리 
# 주말이 없다고 했기 때문에 2020-04-13 ~ 2025-04-14까지 데이터를 생성하여, 국가별 리스트로 새로운 환율 테이블 생성 

res = []
for t in ex['type'].unique() : 
    target = ex[ex['type'] == t]
    # 각 국가별 전체 날짜 생성 : target의 min, max >> 여기서 ex로 가져오지 말기  
    created = pd.date_range(start = target['일자'].min() , end = target['일자'].max()) 
    created = pd.DataFrame({'new_date' : created})
    # 조인을 위해 각 나라의 컬럼 추가 
    created['type'] = t 

    merge = pd.merge(target, created, how = 'right', left_on = ['일자','type'] , right_on = ['new_date' ,'type']) 
    merge['new_date'] = pd.to_datetime(merge['new_date'])
    # 주말 결측을 앞의 데이터로 전체 채우기 
    merge = merge.fillna(method = 'ffill')
    # df와 조인을 위해 Country 생성 
    country_mapper = {
        'jpy-krw' : 'Japan',
        'inr-krw' : 'India',
        'eur-krw' : 'Europe',
        'usd-krw' : 'USA'
    }
    merge['con'] = merge['type'].map(country_mapper)
    res.append(merge)

ex2 = pd.concat(res, ignore_index = True)

In [105]:
# 부품 가격 데이터 전처리
df['timestamp'] = pd.to_datetime(df['timestamp']) 
df['date'] = pd.to_datetime(df['timestamp'].dt.date)

# 환율 날짜의 최소값부터 필터링 
df1 = df[df['date'] >= ex2['new_date'].min()].reset_index(drop = True)

# 주말 여부 - 환율 데이터가 주말이 없기 때문에, 환율 데이터의 날짜 값(일자)이 있으면 "평일 0 " 없으면 "주말 1"
df1['weekend'] = df1['date'].map(lambda x : 0 if x in ex2['일자'].values else 1)
print(df1['weekend'].value_counts())

# 13시 이전/이후 
df1['hour13'] = df1['timestamp'].dt.hour.map(lambda x : 1 if x >= 13 else 0)
df1.head()

weekend
0    16091
1     5757
Name: count, dtype: int64


,timestamp,country,part_category,unit_price,quantity,part_ID,date,weekend,hour13
0,2020-04-13 07:04:00,Japan,Brake,35338.59,45,IDS_6516,2020-04-13,0,0
1,2020-04-13 02:52:00,Japan,Electronics,97732.81,6,IDS_3605,2020-04-13,0,0
2,2020-04-13 05:16:00,India,Engine,10890.93,14,IDS_2209,2020-04-13,0,0
3,2020-04-13 03:07:00,Japan,Brake,24615.55,26,IDS_4124,2020-04-13,0,0
4,2020-04-13 15:55:00,Japan,Suspension,78589.62,4,IDS_4897,2020-04-13,0,1


In [95]:
# 환율 + 부품 가격 합치기 
m_df = pd.merge(df1, ex2 , how = 'left' , left_on = ['date','country'] , right_on = ['new_date', 'con']).reset_index(drop =True)
m_df.head(3)

,timestamp,country,part_category,unit_price,quantity,part_ID,date,weekend,hour13,일자,기준환율,type,new_date,con
0,2020-04-13 07:04:00,Japan,Brake,35338.59,45,IDS_6516,2020-04-13,0,0,2020-04-13,11.1862,jpy-krw,2020-04-13,Japan
1,2020-04-13 02:52:00,Japan,Electronics,97732.81,6,IDS_3605,2020-04-13,0,0,2020-04-13,11.1862,jpy-krw,2020-04-13,Japan
2,2020-04-13 05:16:00,India,Engine,10890.93,14,IDS_2209,2020-04-13,0,0,2020-04-13,15.9800,inr-krw,2020-04-13,India


In [107]:
# 새로운 환율 값 가져오기
# 평일이면 그대로 쓰고, 주말 13시 이후, 다음날 / 주말 13시 이전, 전날 
# 새로 생성한 환율 데이터의 기준환율과 new_date기준 : 포인트) 조건에 해당하는 target 생성으로 

def change_ex(x) : 
    weekend = x['weekend']
    date = x['new_date']     #새로 만든 환율 날짜 기준
    hour = x['hour13']
    country = x['con']       #새로 만든 환율의 나라

    # 환율정보저장
    values = np.nan 

    # 평일(0)일때 
    if weekend == 0 : 
        target = m_df[(m_df['con'] == country) & (m_df['new_date'] == date)]

        if not target.empty : 
            values = target['기준환율'].values[0]

        else : 
            target = m_df[m_df['con'] == country].sort_values('new_date')
            values = target[target['new_date'] <= date]['기준환율'].ffill().iloc[-1]

    # 주말 (1)일떄 
    else : 
        # 13이후일 때 다음 날 
        if hour == 1 : 
            target_date = date + pd.Timedelta(days = 1)
        else : 
            target_date = date - pd.Timedelta(days = 1)

        target = m_df[(m_df['new_date'] == target_date) & (m_df['con'] == country)] 
        # 있으면 그냥 쓰고 
        if not target.empty : 
            values = target['기준환율'].values[0]
        else : 
            # 해당 국가에서 날짜순으로 정렬된 데이터에서 
            target = m_df[m_df['con'] == country].sort_values('new_date')
            # 13시 이후 이면 [다음날]
            if hour == 1 : 
                # 가장 가까운 미래값 bfill의 iloc[0] 첫 번쨰 값 
                values = target[target['new_date'] >= date]['기준환율'].bfill().iloc[0]
            else : 
                # 같거나 이전인 데이터인 경우는 ffill() 이전 환율의 마지막 값 iloc[-1]
                values = target[target['new_date'] <= date]['기준환율'].ffill().iloc[-1]

    return values 

m_df['exchange'] = m_df[['weekend','hour13','con','new_date']].apply(change_ex, axis = 1)


In [108]:
print(m_df.groupby('country')['exchange'].sum())
print(m_df['weekend'].value_counts()) 
print(m_df['hour13'].value_counts()) 

clean_df = m_df[['timestamp','country','part_category','unit_price','quantity','part_ID','hour13','weekend', 'exchange']]

country
Europe    3.099508e+06
India     1.394184e+05
Japan     6.346799e+04
USA       5.627300e+06
Name: exchange, dtype: float64
weekend
0    16091
1     5757
Name: count, dtype: int64
hour13
0    11971
1     9877
Name: count, dtype: int64


In [111]:
# 환율 계산 - country별로 다른 ratio 적용하기 
def cal_price(x) : 
    country = x['country']
    unit = x['unit_price']
    q = x['quantity']
    value = x['exchange']

    ratio = {'Japan' : 3 , 'India' : 5 , 'Europe' : 10, 'USA' : 25}
    return unit * q * (1 + ratio[country] / 100) * value 

clean_df['price'] = clean_df[['country','unit_price','quantity','exchange']].apply(cal_price, axis = 1)
clean_df['price_m'] = clean_df['price'] //  1000000 
clean_df.head(3)

,timestamp,country,part_category,unit_price,quantity,part_ID,hour13,weekend,exchange,price,price_m
0,2020-04-13 07:04:00,Japan,Brake,35338.59,45,IDS_6516,0,0,11.1862,1.832237e+07,18.0
1,2020-04-13 02:52:00,Japan,Electronics,97732.81,6,IDS_3605,0,0,11.1862,6.756339e+06,6.0
2,2020-04-13 05:16:00,India,Engine,10890.93,14,IDS_2209,0,0,15.9800,2.558345e+06,2.0


In [110]:
# 데이터 필터 (주말인 경우만)
w = clean_df[clean_df['weekend'] == 1][['timestamp', 'price' ,'weekend', 'hour13']]
w['price'] = (pd.to_numeric(w['price']) // 1).astype(int)
w

,timestamp,price,weekend,hour13
12,2020-04-15 13:15:00,9172944,1,1
13,2020-04-15 01:06:00,12366163,1,0
14,2020-04-15 11:12:00,7921763,1,0
15,2020-04-15 22:22:00,7259699,1,1
16,2020-04-15 14:20:00,10676431,1,1
...,...,...,...,...
21805,2025-03-22 23:02:00,11711739,1,1
21806,2025-03-22 05:51:00,16442429,1,0
21807,2025-03-22 22:00:00,5565393,1,1
21838,2025-03-29 15:21:00,7376158,1,1


데이터 전처리 (파생변수생성)
* 일자별 발주건들에 대해 국가별 주문수, 카테고리별 종류 수, 카테고리별 제품 수
* 종속변수 : 일본의 수입금액 (독립변수가 존재하지 않는 일자는 제거)
* 시간 관련 파생변수 4개

In [139]:
clean_df['date'] = pd.to_datetime(clean_df['timestamp'].dt.date)
no_japan = clean_df[clean_df['country'] != 'Japan'].reset_index(drop = True)


# 나라, 국가별, 주문 수 
a = no_japan.pivot_table(index = 'date' , 
                         columns = 'country', 
                         aggfunc = 'size').fillna(0).astype(int)

a.columns = a.columns.get_level_values(0)
a = a.reset_index()


country,date,Europe,India,USA
0,2020-04-13,0,2,0
1,2020-04-14,1,1,0
2,2020-04-15,2,3,2
3,2020-04-16,0,1,1
4,2020-04-17,1,3,2
...,...,...,...,...
1522,2025-03-26,1,5,2
1523,2025-03-27,1,1,2
1524,2025-03-28,0,2,0
1525,2025-03-29,0,0,1


In [141]:
# 나라, 국가별, 수입 금액의 갯수와 합계 
b = no_japan.pivot_table(index = 'date', 
                         columns = 'part_category', 
                         values = 'quantity', 
                         aggfunc = ['size','sum']).fillna(0).astype(int)

b.columns = b.columns.get_level_values(1)
b = b.reset_index(drop = True)

In [142]:
f_df = pd.concat([a, b], axis = 1)
f_df.shape

(1527, 14)